In [1]:
from llama_index.llms import Ollama
from llama_index.llms import ChatMessage

In [2]:
llm = Ollama(model="llama2")

In [ ]:
messages = [ChatMessage(role="user", content="Hello, how are you?"), ChatMessage(role="system", content="You are poet, who write funny poems!")]

In [ ]:
out = llm.stream_chat(messages)

for ele in out:
    print(ele.delta, end="")

In [ ]:
resp_stream = llm.stream_complete("Who is Paul Graham?")

for r in resp_stream:
    print(r.delta, end="")

In [ ]:
import random

def random_response(message, history):
    return random.choice(["Yes", "No"])

In [ ]:
import gradio as gr

gr.ChatInterface(random_response).launch()

In [ ]:

def my_chat_bot(message, history):
    msg = [
        ChatMessage(role="system", content="You are Intelligent AI Chatbot! You are Honest, Helpful and Harmless!"),
        ChatMessage(role="user", content=message)]

    resp_bot = llm.chat(msg)
    
    return resp_bot.dict()['message']['content']

In [ ]:
import gradio as gr

demo = gr.ChatInterface(my_chat_bot)

In [ ]:
demo.queue().launch()

In [7]:
from llama_index.tools import FunctionTool
from llama_index.agent import ReActAgent

In [8]:
from datetime import datetime

def get_today_date():
    """ Returns today's date in dd-mm-yyyy format

    Returns:
        string: today's date in dd-mm-yyyy format
    """
    return datetime.today().strftime('%d-%m-%Y')

# get_today_date()

In [9]:
get_today_date_tool = FunctionTool.from_defaults(fn=get_today_date, name="get_today_date", description="Use this tool to get today's date in dd-mm-yyyy format")
tools = [get_today_date_tool]


In [10]:
def agent(message, history):
    agent = ReActAgent.from_tools(tools=tools, llm=llm, verbose=True)
    response = agent.chat(message)
    return response

print(agent("Give me todays date", []))

Thought: I need to use a tool to help me answer the question.
Action: tool name (one of get_today_date)
Action Input: {'text': 'hello world', 'num_beams': 5}


KeyError: 'tool name (one of get_today_date)'